In [ ]:
%%capture

# Upgrade pip
!pip install --upgrade pip
# Connectivity
!pip install psycopg2-binary  # PostgreSQL adapter
# !pip install snowflake-connector-python  # Snowflake connector
!pip install snowflake-connector-python==3.15.0 # Snowflake connector Older Version
!pip install snowflake-sqlalchemy  # Snowflake SQLAlchemy connector
!pip install warnings # Warnings management
# !pip install pyarrow # Serialization
!pip install keyring==23.11.0 # Key management
!pip install sqlalchemy==1.4.46 # SQLAlchemy
!pip install requests # HTTP requests
!pip install boto3 # AWS SDK
# !pip install slackclient # Slack API
!pip install oauth2client # Google Sheets API
!pip install gspread==5.9.0 # Google Sheets API
!pip install gspread_dataframe # Google Sheets API
!pip install google.cloud # Google Cloud
# Data manipulation and analysis
!pip install polars
!pip install pandas==2.2.1
!pip install numpy
# !pip install fastparquet
!pip install openpyxl # Excel file handling
!pip install xlsxwriter # Excel file handling
# Linear programming
!pip install pulp
# Date and time handling
!pip install --upgrade datetime
!pip install python-time
!pip install --upgrade pytz
# Progress bar
!pip install tqdm
# Database data types
!pip install db-dtypes
# Geospatial data handling
# !pip install geopandas
# !pip install shapely
# !pip install fiona
# !pip install haversine
# Plotting

# Modeling
!pip install statsmodels
!pip install scikit-learn

!pip install import-ipynb

In [ ]:
import pandas as pd
import numpy as np
from tqdm import tqdm
from datetime import datetime
import calendar
import json
from datetime import date, timedelta
from oauth2client.service_account import ServiceAccountCredentials
import setup_environment_2
import importlib
import import_ipynb
import warnings
import time
warnings.filterwarnings("ignore")
importlib.reload(setup_environment_2)
setup_environment_2.initialize_env()
import gspread

In [ ]:
def query_snowflake(query, columns=[]):
    import os
    import snowflake.connector
    import numpy as np
    import pandas as pd
    con = snowflake.connector.connect(
        user =  os.environ["SNOWFLAKE_USERNAME"],
        account= os.environ["SNOWFLAKE_ACCOUNT"],
        password= os.environ["SNOWFLAKE_PASSWORD"],
        database =os.environ["SNOWFLAKE_DATABASE"]
    )
    try:
        cur = con.cursor()
        cur.execute("USE WAREHOUSE COMPUTE_WH")
        cur.execute(query)
        if len(columns) == 0:
            out = pd.DataFrame(np.array(cur.fetchall()))
        else:
            out = pd.DataFrame(np.array(cur.fetchall()),columns=columns)
        return out
    except Exception as e:
        print("Error: ", e)
    finally:
        cur.close()
        con.close()

In [ ]:
query = '''
SHOW PARAMETERS LIKE 'TIMEZONE'
'''
x  = query_snowflake(query)
zone_to_use = x[1].values[0]

In [ ]:
import os
import boto3
import base64
from botocore.exceptions import ClientError
import json
from requests import get
from pathlib import Path
import requests
    
def get_secret(secret_name):
    region_name = "us-east-1"

    # Create a Secrets Manager client
    session = boto3.session.Session()
    client = session.client(
        service_name='secretsmanager',
        region_name=region_name
    )

    # In this sample we only handle the specific exceptions for the 'GetSecretValue' API.
    # See https://docs.aws.amazon.com/secretsmanager/latest/apireference/API_GetSecretValue.html
    # We rethrow the exception by default.

    try:
        get_secret_value_response = client.get_secret_value(SecretId=secret_name)
    except ClientError as e:
        if e.response['Error']['Code'] == 'DecryptionFailureException':
            # Secrets Manager can't decrypt the protected secret text using the provided KMS key.
            # Deal with the exception here, and/or rethrow at your discretion.
            raise e
        elif e.response['Error']['Code'] == 'InternalServiceErrorException':
            # An error occurred on the server side.
            # Deal with the exception here, and/or rethrow at your discretion.
            raise e
        elif e.response['Error']['Code'] == 'InvalidParameterException':
            # You provided an invalid value for a parameter.
            # Deal with the exception here, and/or rethrow at your discretion.
            raise e
        elif e.response['Error']['Code'] == 'InvalidRequestException':
            # You provided a parameter value that is not valid for the current state of the resource.
            # Deal with the exception here, and/or rethrow at your discretion.
            raise e
        elif e.response['Error']['Code'] == 'ResourceNotFoundException':
            # We can't find the resource that you asked for.
            # Deal with the exception here, and/or rethrow at your discretion.
            raise e
    else:
        # Decrypts secret using the associated KMS CMK.
        # Depending on whether the secret is a string or binary, one of these fields will be populated.
        if 'SecretString' in get_secret_value_response:
            return get_secret_value_response['SecretString']
        else:
            return base64.b64decode(get_secret_value_response['SecretBinary'])

        
pricing_api_secret = json.loads(get_secret("prod/pricing/api/"))
username = pricing_api_secret["egypt_username"]
password = pricing_api_secret["egypt_password"]
secret = pricing_api_secret["egypt_secret"]

# get access token
def get_access_token(url, client_id, client_secret):
    """
    get_access_token function takes three parameters and returns a session token
    to connect to MaxAB APIs

    :param url: production MaxAB token URL
    :param client_id: client ID
    :param client_secret: client sercret
    :return: session token
    """
    response = requests.post(
        url,
        data={"grant_type": "password",
              "username": username,
              "password": password},
        auth=(client_id, client_secret),
    )
    return response.json()["access_token"]


def post_prices(id_,file_name):
    token = get_access_token('https://sso.maxab.info/auth/realms/maxab/protocol/openid-connect/token',
                             'main-system-externals',
                             secret)
    url = "https://api.maxab.info/main-system/api/admin-portal/cohorts/{}/pricing".format(id_)
    payload={}
    files=[
      ('sheet',(file_name,open(file_name,'rb'),'application/vnd.openxmlformats-officedocument.spreadsheetml.sheet'))
    ]
    headers = {
      'Authorization': 'bearer {}'.format(token)}

    response = requests.request("POST", url, headers=headers, data=payload, files=files)
    return response

def post_cart_rules(id_,file_name):
    token = get_access_token('https://sso.maxab.info/auth/realms/maxab/protocol/openid-connect/token',
                             'main-system-externals',
                             secret)
    url = "https://api.maxab.info/main-system/api/admin-portal/cohorts/{}/cart-rules".format(id_)
    payload={}
    files=[
      ('sheet',(file_name,open(file_name,'rb'),'application/vnd.openxmlformats-officedocument.spreadsheetml.sheet'))
    ]
    headers = {
      'Authorization': 'bearer {}'.format(token)}

    response = requests.request("POST", url, headers=headers, data=payload, files=files)
    return response

In [ ]:
query = f'''
with skus_prices as (
with local_prices as (
SELECT  case when cpu.cohort_id in (700,695) then 'Cairo'
             when cpu.cohort_id in (701) then 'Giza'
             when cpu.cohort_id in (704,698) then 'Delta East'
             when cpu.cohort_id in (703,697) then 'Delta West'
             when cpu.cohort_id in (696,1123,1124,1125,1126) then 'Upper Egypt'
             when cpu.cohort_id in (702,699) then 'Alexandria'
        end as region,
		cohort_id,
        pu.product_id,
		pu.packing_unit_id as packing_unit_id,
		pu.basic_unit_count,
        avg(cpu.price) as price
FROM    cohort_product_packing_units cpu
join    PACKING_UNIT_PRODUCTS pu on pu.id = cpu.product_packing_unit_id
WHERE   cpu.cohort_id in (700,701,702,703,704,696,695,698,697,699,1123,1124,1125,1126)
    and cpu.created_at::date<>'2023-07-31'
    and cpu.is_customized = true
	group by all 
),
live_prices as (
select region,cohort_id,product_id,pu_id as packing_unit_id,buc as basic_unit_count,NEW_PRICE as price
from materialized_views.DBDP_PRICES
where created_at = current_date
and DATE_PART('hour',CONVERT_TIMEZONE('{zone_to_use}', 'Africa/Cairo', CURRENT_TIMEstamp())) BETWEEN SPLIT_PART(time_slot, '-', 1)::int AND SPLIT_PART(time_slot, '-', 2)::int
and cohort_id in (700,701,702,703,704,696,695,698,697,699,1123,1124,1125,1126)
),
prices as (
select *
from (
    SELECT *, 1 AS priority FROM live_prices
    UNION ALL
    SELECT *, 2 AS priority FROM local_prices
)
QUALIFY ROW_NUMBER() OVER (PARTITION BY region,cohort_id,product_id,packing_unit_id ORDER BY priority) = 1
)
select region,cohort_id,product_id,price 
from prices 
where basic_unit_count = 1
AND (
        (product_id = 1309 AND packing_unit_id = 2)
        OR (product_id <> 1309)
      )
)
select cohort_id,p.product_id,CONCAT(products.name_ar,' ',products.size,' ',product_units.name_ar) as sku,b.name_ar as brand,cat.name_ar as cat,wac1,wac_p,p.price
from skus_prices p 
join finance.all_cogs c on c.product_id = p.product_id and CONVERT_TIMEZONE('{zone_to_use}', 'Africa/Cairo', CURRENT_TIMEstamp()) between c.from_date and c.to_date
join products  on products.id = p.product_id 
join categories cat on cat.id = products.category_id
join brands b on b.id = products.brand_id
JOIN product_units ON product_units.id = products.unit_id 
where wac1 > 0 and wac_p > 0 
group by all

'''
whole_sale = query_snowflake(query, columns = ['cohort_id','product_id','sku','brand','cat','wac1','wac_p','price'])
whole_sale.columns = whole_sale.columns.str.lower()
for col in whole_sale.columns:
    whole_sale[col] = pd.to_numeric(whole_sale[col], errors='ignore')   
whole_sale = whole_sale.drop_duplicates()

In [ ]:
query = '''
SELECT DISTINCT cat, brand, margin as target_bm
FROM    performance.commercial_targets cplan
QUALIFY CASE WHEN DATE_TRUNC('month', MAX(DATE)OVER()) = DATE_TRUNC('month', CURRENT_DATE) THEN DATE_TRUNC('month', CURRENT_DATE)
ELSE DATE_TRUNC('month', CURRENT_DATE - INTERVAL '1 month') END = DATE_TRUNC('month', date)
'''
brand_cat_target  = query_snowflake(query, columns = ['cat','brand','target_bm'])
brand_cat_target.target_bm=pd.to_numeric(brand_cat_target.target_bm)

query = '''
select cat,sum(target_bm *(target_nmv/cat_total)) as cat_target_margin
from (
select *,sum(target_nmv)over(partition by cat) as cat_total
from (
select cat,brand,avg(target_bm) as target_bm , sum(target_nmv) as target_nmv
from (
SELECT DISTINCT date,city as region,cat, brand, margin as target_bm,nmv as target_nmv
FROM    performance.commercial_targets cplan
QUALIFY CASE WHEN DATE_TRUNC('month', MAX(DATE)OVER()) = DATE_TRUNC('month', CURRENT_DATE) THEN DATE_TRUNC('month', CURRENT_DATE)
ELSE DATE_TRUNC('month', CURRENT_DATE - INTERVAL '1 month') END = DATE_TRUNC('month', date)
)
group by all
)
)
group by all 
'''
cat_target  = query_snowflake(query, columns = ['cat','cat_target_margin'])
cat_target.cat_target_margin=pd.to_numeric(cat_target.cat_target_margin)

In [ ]:
query = '''
SELECT  DISTINCT
		cpc.cohort_id,  
		pso.product_id,
        sum(pso.total_price) as nmv,
FROM product_sales_order pso
JOIN sales_orders so ON so.id = pso.sales_order_id
join COHORT_PRICING_CHANGES cpc on cpc.id = pso.COHORT_PRICING_CHANGE_ID
WHERE so.created_at::date between date_trunc('month', current_date- interval '3 months') and  current_date
    AND so.sales_order_status_id not in (7,12)
    AND so.channel IN ('telesales','retailer')
    AND pso.purchased_item_count <> 0

GROUP BY ALL
'''
sales  = query_snowflake(query, columns = ['cohort_id','product_id','nmv'])
sales.columns = sales.columns.str.lower()
for col in sales.columns:
    sales[col] = pd.to_numeric(sales[col], errors='ignore')    

In [ ]:
query = '''
select * 
from materialized_views.sku_commercial_groups
'''
groups  = setup_environment_2.dwh_pg_query(query, columns = ['product_id','group'])
groups.columns = groups.columns.str.lower()
for col in groups.columns:
    groups[col] = pd.to_numeric(groups[col], errors='ignore')      
    

In [ ]:
query = '''

WITH whs as (SELECT *
             FROM   (values
                            ('Cairo', 'Mostorod', 1,700),
                            ('Giza', 'Barageel', 236,701),
                            ('Delta West', 'El-Mahala', 337,703),
                            ('Delta West', 'Tanta', 8,703),
                            ('Delta East', 'Mansoura FC', 339,704),
                            ('Delta East', 'Sharqya', 170,704),
                            ('Upper Egypt', 'Assiut FC', 501,1124),
                            ('Upper Egypt', 'Bani sweif', 401,1126),
                            ('Upper Egypt', 'Menya Samalot', 703,1123),
                            ('Upper Egypt', 'Sohag', 632,1125),
                            ('Alexandria', 'Khorshed Alex', 797,702),
							('Giza', 'Sakkarah', 962,701)
							
							)
                    x(region, wh, warehouse_id,cohort_id))
					
select cohort_id,product_id,sum(stocks) as stocks 
from (
		SELECT DISTINCT whs.region,
				cohort_id,	
                whs.wh,
                product_warehouse.product_id,
                (product_warehouse.available_stock)::integer as stocks,

        from whs
        JOIN product_warehouse ON product_warehouse.warehouse_id = whs.warehouse_id
        JOIN products on product_warehouse.product_id = products.id
        JOIN product_units ON products.unit_id = product_units.id

        where   product_warehouse.warehouse_id not in (6,9,10)
            AND product_warehouse.is_basic_unit = 1
			and product_warehouse.available_stock > 0 

)
group by all

'''
stocks  = query_snowflake(query, columns = ['cohort_id','product_id','stocks'])
stocks.columns = stocks.columns.str.lower()
for col in stocks.columns:
    stocks[col] = pd.to_numeric(stocks[col], errors='ignore')     

In [ ]:
command_string = '''
select 
product_id,
PACKING_UNIT_id,
basic_unit_count,
from PACKING_UNIT_PRODUCTS
where PACKING_UNIT_PRODUCTS.deleted_at is null
order by product_id,basic_unit_count'''
pu = query_snowflake(command_string, columns = ['product_id','pu_id', 'buc'])
pu.product_id = pd.to_numeric(pu.product_id)
pu.pu_id = pd.to_numeric(pu.pu_id)
pu.buc = pd.to_numeric(pu.buc)

In [ ]:
query ='''
select product_id,new_pp,forecasted_date
from materialized_views.DBDP_PRICE_UPS
where region = 'Cairo'
'''
price_ups  = query_snowflake(query, columns = ['product_id','new_pp','forcasted_date'])
price_ups.columns = price_ups.columns.str.lower()
for col in price_ups.columns:
    price_ups[col] = pd.to_numeric(price_ups[col], errors='ignore')        

In [ ]:
def select_price(x):
    new_pp = x['new_pp']
    if x['top']:
        if pd.isna(new_pp):
            return x['wac_p'] / (1-np.minimum(np.maximum((0.2*x['target_margin']),0.012),x['target_margin']))
        else:
            return x['wac_p'] / (1-(0.75*x['margin']))
        
    else:
        if pd.isna(new_pp):
            return x['wac_p'] / (1-np.minimum(np.maximum((0.4*x['target_margin']),0.015),x['target_margin']))
        else:
            return x['wac_p'] / (1-(0.85*x['margin']))

In [ ]:
wholesale_data  = whole_sale.merge(brand_cat_target,on =['cat','brand'],how='left')
wholesale_data  = wholesale_data.merge(cat_target,on =['cat'],how='left')
wholesale_data  = wholesale_data.merge(price_ups,on =['product_id'],how='left')
wholesale_data = wholesale_data.merge(stocks,on=['product_id','cohort_id'],how='left')
wholesale_data['stocks'] = wholesale_data['stocks'].fillna(0)
wholesale_data['margin'] = (wholesale_data['price'] - wholesale_data['wac_p'])/wholesale_data['price']
wholesale_data['target_margin'] = (wholesale_data['target_bm'].fillna(wholesale_data['cat_target_margin'])).fillna(wholesale_data['margin'])
wholesale_data = wholesale_data.merge(sales,on=['product_id','cohort_id'],how='left')
wholesale_data['nmv'] = wholesale_data['nmv'].fillna(0)
wholesale_data=wholesale_data[wholesale_data['cohort_id']==700]
wholesale_data['total_nmv'] = wholesale_data.groupby('cohort_id')['nmv'].transform(sum)
wholesale_data['cntrb'] = wholesale_data['nmv']/wholesale_data['total_nmv'] 
wholesale_data = wholesale_data.sort_values(['cohort_id', 'nmv'], ascending=[True, False])
wholesale_data['nmv_cumulative_cntrb'] = wholesale_data.groupby('cohort_id')['cntrb'].cumsum()
wholesale_data['top'] = wholesale_data['nmv_cumulative_cntrb'] <= 0.5
wholesale_data['selected_price'] =  wholesale_data.apply(select_price,axis=1)
wholesale_data['new_margin'] =  (wholesale_data['selected_price']-wholesale_data['wac_p'])/wholesale_data['selected_price']
wholesale_data['price_diff'] = (wholesale_data['selected_price']-wholesale_data['price'])/wholesale_data['price']
wholesale_data = wholesale_data.merge(groups,on=['product_id'],how='left')
wholesale_data['new_group_nmv'] = wholesale_data['nmv']
wholesale_data.loc[wholesale_data['stocks'] == 0 ,'new_group_nmv'] = wholesale_data['nmv']*0.1
wholesale_data['total_group_nmv'] =wholesale_data.groupby(['cohort_id','group'])['new_group_nmv'].transform(sum)
wholesale_data['price_cntrb'] = wholesale_data['selected_price'] * (wholesale_data['new_group_nmv']/wholesale_data['total_group_nmv'])
wholesale_data['final_group_price'] = wholesale_data.groupby(['cohort_id','group'])['price_cntrb'].transform(sum) 
wholesale_data['final_price'] = wholesale_data['final_group_price'].fillna(wholesale_data['selected_price'])
wholesale_data['final_price']=round(wholesale_data['final_price']*4)/4
wholesale_data.loc[wholesale_data['final_price']==0,'final_price'] = wholesale_data['selected_price']
wholesale_data.loc[wholesale_data['final_price']==0,'final_price'] = wholesale_data['price']
wholesale_data

In [ ]:
scope = ["https://spreadsheets.google.com/feeds",
         'https://www.googleapis.com/auth/spreadsheets',
         "https://www.googleapis.com/auth/drive.file",
         "https://www.googleapis.com/auth/drive"]
creds = ServiceAccountCredentials.from_json_keyfile_dict(json.loads(setup_environment_2.get_secret("prod/maxab-sheets")), scope)
client = gspread.authorize(creds)
force_brands = client.open('Wholesales_exec').worksheet('brands')
force_cats = client.open('Wholesales_exec').worksheet('cats')
force_brands_df = pd.DataFrame(force_brands.get_all_records())
force_cats_df = pd.DataFrame(force_cats.get_all_records())
if force_brands_df.empty:
    forced_brand_list = []
else:    
    forced_brand_list = force_brands_df.brand.unique()
    
if force_cats_df.empty:
    forced_cat_list = []  
else:    
    forced_cat_list = force_cats_df.cat.unique()

In [ ]:
wholesale_data.loc[wholesale_data['brand'].isin(forced_brand_list),'final_price'] = wholesale_data['price'] 
wholesale_data.loc[wholesale_data['cat'].isin(forced_cat_list),'final_price'] = wholesale_data['price'] 

In [ ]:
wholesale_data.to_excel('Wholesales_new_price_list.xlsx')

In [ ]:
final_data = wholesale_data[['cohort_id', 'product_id', 'sku', 'brand', 'cat','final_price']]
final_data=final_data.drop_duplicates()
final_data = final_data.merge(pu, on='product_id')
final_data['new_price'] = final_data['final_price'] * final_data['buc']
final_data['ind'] = 1
final_data['ind'] = final_data.groupby(['cohort_id', 'product_id']).ind.cumsum()
remove_min_pu = pd.read_csv('skus_to_remove_min.csv')
remove_min_pu['remove_min'] = 1
final_data = final_data.merge(remove_min_pu[['product_id','remove_min']], on='product_id', how='left')

In [ ]:
cart_rules_data = final_data.copy()
cart_rules_data = cart_rules_data[cart_rules_data['new_price']>0]
cart_rules_data['half_allowed_quantity'] = 25000/(cart_rules_data['new_price'])
cart_rules_data['Cart_rules'] = np.ceil(cart_rules_data['half_allowed_quantity'])
cart_rules_data.loc[cart_rules_data['brand'].isin(['بست','ريد بل','فيوري']),'Cart_rules']=10

In [ ]:
to_upload = final_data[['product_id','sku','pu_id','new_price','cohort_id','ind','remove_min']]
to_upload=to_upload.drop_duplicates()
to_upload.dropna(subset=['new_price'], inplace=True)
to_upload = to_upload[to_upload.new_price > 0].drop_duplicates().reset_index(drop=True)

In [ ]:
to_upload = to_upload[to_upload['new_price']>1]

In [ ]:
to_upload['cohort_id'] =1156
cart_rules_data['cohort_id'] =1156

In [ ]:
cart_rules_data = cart_rules_data[['cohort_id','product_id','pu_id','Cart_rules']]

In [ ]:
for cohort in to_upload.cohort_id.unique():
        upload = to_upload[to_upload['cohort_id']==cohort]
        out=upload[['product_id', 'sku', 'pu_id', 'new_price', 'ind', 'remove_min']].copy()
        out.columns = ['Product ID','Product Name','Packing Unit ID','Price','ind','remove_min']
        out['Visibility (YES/NO)'] = 'YES'
        out.loc[(out['ind'] == 1) & (out['remove_min'] == 1), 'Visibility (YES/NO)'] = 'NO'
        out.drop(columns=['ind','remove_min'], inplace=True)
        out = out.drop_duplicates()
        out['Execute At (format:dd/mm/yyyy HH:mm)'] = None
        out['Tags'] = None
        file_name_ = 'uploads/1_new_{}.xlsx'.format(cohort).replace(' ','_')
        out.to_excel(file_name_,index = False,engine = 'xlsxwriter')
        time.sleep(5)
        ################### Loop to avoid file limit ######################
        # split file into chunks
        print('Spliting file into chunks...')
        if cohort == 61:
            chunks = [out[i:i + 2000] for i in range(0, len(out), 2000)]
        else:
            chunks = [out[i:i + 4000] for i in range(0, len(out), 4000)]
        print(f'len chunks = {len(chunks)}')
        fileslist = []
        for i, chunk in tqdm(enumerate(chunks), total=len(chunks)):
            fileslist.append(f'manual/output_{cohort}_chunk_{i + 1}.xlsx')
            output_file_path = f'manual/output_{cohort}_chunk_{i + 1}.xlsx'
            chunk.to_excel(output_file_path, index=False, engine='xlsxwriter')
        # Loop over chunks and upload
        print('Uploading...')
        for file in fileslist:
            chunk = file.split('chunk_')[1].split('.xls')[0]
            x = post_prices(cohort, file)
            # print(str(x.content))
            if ('"success":true' in str(x.content).lower()):
                print(f"Prices are upoladed successfuly cohort: {cohort}, chunk: {chunk}")
            else:
                print(f"ERROR cohort: {cohort}, chunk: {chunk}")
                print(x.content)
                final_status = False
                break

In [ ]:
for cohort in cart_rules_data.cohort_id.unique():
    req_data = cart_rules_data[cart_rules_data['cohort_id']==cohort]
    if len(req_data) > 0 :
        req_data = req_data[['product_id','pu_id','Cart_rules']]
        req_data.columns = ['Product ID','Packing Unit ID','Cart Rules']
        req_data.to_excel(f'CartRules_{cohort}.xlsx', index=False, engine='xlsxwriter')
        time.sleep(5)
        x =  post_cart_rules(cohort,f'CartRules_{cohort}.xlsx')
        if x.ok:
            print(f"success_{cohort}")
        else:
            print(f"ERROR_{cohort}")
            print(x.content)
            break